In [1]:
import pathlib
from typing import List

import scipy.stats
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KDTree
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, roc_curve
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
PATH_DATA = "./../data/interim"
PATH_MODELS = "./../models"
PATH_MATCH_DATA = "./../data/raw/cms.csv"
PATH_ROTTERDAM = "./../data/raw/cms_rotterdam.csv"

CMS_FREQUENCY = 0.146

THRESHOLDS = {
    "ANN": 0.5,
    "LR": 0.5,
    "BAE": 100,
    "LIU": 238,
}

CALIBRATE_PROBABILITY = True

In [3]:
def calibrate_proba(
    X: np.ndarray, 
    true_frequency: float, 
    observed_frequency: float,
) -> np.ndarray:
    """Dal Pozzolo A, Caelen O, Johnson RA, Bontempi G. Calibrating probability
    with undersampling for unbalanced classification. In2015 IEEE Symposium
    Series on Computational Intelligence 2015 Dec 7 (pp. 159-166). IEEE."""
    return (X * true_frequency / observed_frequency) / (
        (
            ((1 - X) * (1 - true_frequency) / (1 - observed_frequency))
            + (X * true_frequency / observed_frequency)
        )
    )

In [4]:
def get_sample_weights(y: np.ndarray) -> np.ndarray:
    observed_frequency = np.count_nonzero(y) / len(y)
    return np.where(
        y,
        CMS_FREQUENCY / observed_frequency,
        (1 - CMS_FREQUENCY) / (1 - observed_frequency),
    )


def accuracy_score_weighted(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return accuracy_score(y_true, y_pred, sample_weight=get_sample_weights(y_true))


def recall_score_weighted(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return recall_score(y_true, y_pred, sample_weight=get_sample_weights(y_true))


def precision_score_weighted(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return precision_score(
        y_true, y_pred, sample_weight=get_sample_weights(y_true), zero_division=0
    )


def roc_auc_score_weighted(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return roc_auc_score(y_true, y_pred, sample_weight=get_sample_weights(y_true))


def roc_curve_weighted(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return roc_curve(y_true, y_pred, sample_weight=get_sample_weights(y_true))


def specificity_score_weighted(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return recall_score(
        y_true, y_pred, pos_label=0, sample_weight=get_sample_weights(y_true)
    )


def npv_score_weighted(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return precision_score(
        y_true,
        y_pred,
        pos_label=0,
        sample_weight=get_sample_weights(y_true),
        zero_division=0,
    )


In [5]:
match_tree = KDTree(pd.read_csv(PATH_MATCH_DATA, header=None).to_numpy())
rotterdam_scores = pd.read_csv(PATH_ROTTERDAM)

In [6]:
METRICS = {
    "AUC": (roc_auc_score_weighted, False),
    "Accuracy": (accuracy_score_weighted, True),
    "Sensitivity": (recall_score_weighted, True),
    "Specificity": (specificity_score_weighted, True),
    "PPV": (precision_score_weighted, True),
    "NPV": (npv_score_weighted, True),
    "_ROC": (roc_curve_weighted, False),
}

results = {}

for split_index in [i for s in range(10) for i in list(range(s, 100, 10))]:
    path_pred, path_test, path_train = (
        pathlib.Path(PATH_MODELS) / f"predict-ann-{split_index}.csv",
        pathlib.Path(PATH_DATA) / f"test-{split_index}.csv",
        pathlib.Path(PATH_DATA) / f"train-{split_index}.csv",
    )

    if (
        not pathlib.Path(path_pred).is_file()
        or not pathlib.Path(path_test).is_file()
        or not pathlib.Path(path_train).is_file()
    ):
        continue

    df_test = pd.read_csv(path_test)
    df_pred = pd.read_csv(path_pred)

    y_pred = {}
    y_pred["ANN"] = df_pred["predict0"]

    df_train = pd.read_csv(path_train)
    output_column_mask = df_train.columns.isin({"output0"})
    X_train, y_train = (
        df_train.iloc[:, ~output_column_mask],
        df_train.iloc[:, output_column_mask],
    )
    X_train, y_train = (
        X_train.to_numpy(np.float64),
        y_train.to_numpy(np.bool8)[:, 0],
    )
    X_test = df_test.iloc[:, ~output_column_mask].to_numpy(np.float64)

    for model_name, model_cls in [
        ("LR", LogisticRegression),
    ]:
        model = model_cls()
        model.fit(X_train, y_train)
        y_pred[model_name] = model.predict_proba(X_test)[:, 1]

    y_true = df_test["output0"].to_numpy(np.bool8)

    if CALIBRATE_PROBABILITY:
        df_train = pd.read_csv(path_train)
        observed_frequency = np.count_nonzero(df_train["output0"]) / len(df_train)

        for model in y_pred:
            y_pred[model] = calibrate_proba(
                y_pred[model], CMS_FREQUENCY, observed_frequency
            )

    y_pred.update(
        {
            name: series.to_numpy()
            for name, series in rotterdam_scores.iloc[
                np.ravel(match_tree.query(X_test, k=1)[1])
            ].items()
        }
    )

    for model in y_pred:
        if model not in results:
            results[model] = {}

        for name, (func, threshold) in METRICS.items():
            results[model][name] = results[model].get(name, []) + [
                func(
                    y_true,
                    y_pred[model] >= THRESHOLDS[model] if threshold else y_pred[model],
                )
            ]

In [7]:
df = pd.DataFrame.from_records(results)
df = df[~df.index.str.startswith("_")]
df.columns = pd.MultiIndex.from_arrays(
    [
        ["Rotterdam" if col in {"BAE", "LIU"} else "Ours" for col in df.columns],
        df.columns,
    ]
)
df = df.reindex(sorted(df.columns), axis=1)

df_median = df.applymap(lambda x: np.median(x))
df_median.index = pd.MultiIndex.from_product([["Median"], df_median.index])
df_mean = df.applymap(lambda x: np.mean(x))
df_mean.index = pd.MultiIndex.from_product([["Mean"], df_mean.index])

df_style = (
    pd.concat([df_mean, df_median], axis=0)
    .style.set_caption(
        "Comparing classification performance of different models predicting "
        "cerebellar mutism syndrome using 10x10 stratified cross-validation. "
        "Results are shown with both mean and median averages. <br/><br/>"
        "ANN=Artificial neural network, LR=Logistic regression, "
        "AUC=Area under curve"
    )
    .format(lambda v: "-" if v == 0 else f"{v:.3f}", na_rep="")
    .highlight_max(color="lightblue", axis=1)
)
display(df_style)

In [8]:
def print_metric_significance(
    models: List[str],
    star_scale: List[float] = [0.05, 0.01, 0.001],
    cutoff: float = 0.001,
):
    for metric in METRICS:
        if metric.startswith("_"):
            continue
        metric_ann, metric_lr = df.loc[metric, models]
        #         p_value = scipy.stats.ttest_rel(metric_ann, metric_lr).pvalue
        p_value = scipy.stats.wilcoxon(metric_ann, metric_lr).pvalue
        print(
            f"{metric: >12}: "
            f"{'  ' if p_value > cutoff else '< '}{max(p_value, cutoff):.3f} "
            f"{''.join('*' if p_value <= scale else '' for scale in star_scale)}"
        )
    print()


df = pd.DataFrame.from_records(results)

print(f"Number of runs: {len(df.iloc[0, 0])}\n")

for model_name in results:
    if model_name == "ANN":
        continue
    print(f"ANN vs {model_name}")
    print_metric_significance(["ANN", model_name])


Number of runs: 100

ANN vs LR
         AUC:   0.006 **
    Accuracy:   0.055 
 Sensitivity: < 0.001 ***
 Specificity:   0.737 
         PPV:   0.828 
         NPV: < 0.001 ***

ANN vs BAE
         AUC: < 0.001 ***
    Accuracy: < 0.001 ***
 Sensitivity: < 0.001 ***
 Specificity: < 0.001 ***
         PPV: < 0.001 ***
         NPV: < 0.001 ***

ANN vs LIU
         AUC: < 0.001 ***
    Accuracy: < 0.001 ***
 Sensitivity: < 0.001 ***
 Specificity: < 0.001 ***
         PPV: < 0.001 ***
         NPV: < 0.001 ***



In [15]:
def setup_roc_plot(figure_size):
    plt.figure(figsize=figure_size)
    ax = plt.axes()
    ax.set_aspect("equal")
    ax.set_axisbelow(True)
    plt.xlabel("1-Specificity (False Positive Rate)")
    plt.ylabel("Sensitivity (True Positive Rate)")
    plt.title("Receiver Operating Characteristic")
    return ax


unique_fpr = []
for model_name in results:
    for fpr, tpr, thrsholds in results[model_name]["_ROC"]:
        unique_fpr += list(fpr)
unique_fpr = np.unique(unique_fpr)

with plt.rc_context({"font.size": 16}):
    ax = setup_roc_plot(figure_size=(8, 8))

    for model_name in results:
        mean_tpr = np.zeros((len(results[model_name]["_ROC"]), len(unique_fpr)))
        for index, (fpr, tpr, thrsholds) in enumerate(results[model_name]["_ROC"]):
            mean_tpr[index] = np.interp(unique_fpr, fpr, tpr)
        mean_tpr = np.mean(mean_tpr, axis=0)
        ax.plot(
            unique_fpr,
            mean_tpr,
            linewidth=3,
            label=f"{model_name} (AUC={np.mean(results[model_name]['AUC']):.3f})",
        )

    plt.legend(loc="lower right")
    plt.show()